# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 29 2022 8:30AM,247519,10,WT5357 Out,Eywa Pharma Inc.,Released
1,Sep 29 2022 8:27AM,247526,19,60017718,"GUSA Granules USA, Inc.",Released
2,Sep 29 2022 8:25AM,247520,20,ATR-I000008,"HVL, LLC dba Atrium Innovations",Released
3,Sep 29 2022 8:08AM,247525,19,MSP214428,"Methapharm, Inc.",Released
4,Sep 29 2022 8:08AM,247525,19,MSP214429,"Methapharm, Inc.",Released
5,Sep 29 2022 8:08AM,247525,19,MSP214434,"Methapharm, Inc.",Released
6,Sep 29 2022 8:04AM,247524,19,ADV80006078,"AdvaGen Pharma, Ltd",Released
7,Sep 29 2022 8:04AM,247524,19,ADV80006079,"AdvaGen Pharma, Ltd",Released
8,Sep 29 2022 8:04AM,247524,19,ADV80006081,"AdvaGen Pharma, Ltd",Released
9,Sep 29 2022 8:04AM,247524,19,ADV80006082,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,247520,Released,1
10,247523,Released,2
11,247524,Released,18
12,247525,Released,3
13,247526,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
247520,NaN,1.0
247523,NaN,2.0
247524,NaN,18.0
247525,NaN,3.0
247526,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
247395,0.0,1.0
247471,0.0,1.0
247480,0.0,1.0
247489,0.0,1.0
247490,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
247395,0,1
247471,0,1
247480,0,1
247489,0,1
247490,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,247395,0,1
1,247471,0,1
2,247480,0,1
3,247489,0,1
4,247490,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,247395,,1
1,247471,,1
2,247480,,1
3,247489,,1
4,247490,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.
1,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc."
2,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations"
3,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc."
6,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd"
24,Sep 29 2022 8:03AM,247523,10,Emerginnova
26,Sep 28 2022 6:05PM,247490,21,"NBTY Global, Inc."
27,Sep 28 2022 6:05PM,247395,21,"NBTY Global, Inc."
28,Sep 28 2022 6:03PM,247496,21,"NBTY Global, Inc."
29,Sep 28 2022 5:59PM,247489,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.,,1
1,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc.",,1
2,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations",,1
3,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc.",,3
4,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd",,18
5,Sep 29 2022 8:03AM,247523,10,Emerginnova,,2
6,Sep 28 2022 6:05PM,247490,21,"NBTY Global, Inc.",,1
7,Sep 28 2022 6:05PM,247395,21,"NBTY Global, Inc.",,1
8,Sep 28 2022 6:03PM,247496,21,"NBTY Global, Inc.",,1
9,Sep 28 2022 5:59PM,247489,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.,1,
1,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc.",1,
2,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations",1,
3,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc.",3,
4,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd",18,
5,Sep 29 2022 8:03AM,247523,10,Emerginnova,2,
6,Sep 28 2022 6:05PM,247490,21,"NBTY Global, Inc.",1,
7,Sep 28 2022 6:05PM,247395,21,"NBTY Global, Inc.",1,
8,Sep 28 2022 6:03PM,247496,21,"NBTY Global, Inc.",1,
9,Sep 28 2022 5:59PM,247489,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.,1,
1,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc.",1,
2,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations",1,
3,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc.",3,
4,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd",18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.,1,NaN
1,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc.",1,NaN
2,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations",1,NaN
3,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc.",3,NaN
4,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd",18,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 29 2022 8:30AM,247519,10,Eywa Pharma Inc.,1,0.0
1,Sep 29 2022 8:27AM,247526,19,"GUSA Granules USA, Inc.",1,0.0
2,Sep 29 2022 8:25AM,247520,20,"HVL, LLC dba Atrium Innovations",1,0.0
3,Sep 29 2022 8:08AM,247525,19,"Methapharm, Inc.",3,0.0
4,Sep 29 2022 8:04AM,247524,19,"AdvaGen Pharma, Ltd",18,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,990022,5,10.0
16,247480,1,0.0
19,742575,22,0.0
20,247520,1,0.0
21,989870,4,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,990022,5,10.0
1,16,247480,1,0.0
2,19,742575,22,0.0
3,20,247520,1,0.0
4,21,989870,4,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,5,10.0
1,16,1,0.0
2,19,22,0.0
3,20,1,0.0
4,21,4,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,5.0
1,16,Released,1.0
2,19,Released,22.0
3,20,Released,1.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,10.0,0.0,0.0,0.0,0.0
Released,5.0,1.0,22.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0
1,Released,5.0,1.0,22.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0
1,Released,5.0,1.0,22.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()